# Оценка эффективности промптов

## Обзор
В этом уроке рассматриваются методы и техники оценки эффективности промптов для языковых моделей. Разбираются метрики качества, а также ручная и автоматизированная оценка.

## Мотивация
Чтобы улучшать промпты, нужно уметь измерять их качество. Систематическая оценка позволяет находить наиболее эффективные формулировки и получать более надёжные ответы от модели.

## Ключевые компоненты
1. Метрики качества промптов.
2. Ручная оценка.
3. Автоматизированная оценка.
4. Практические примеры с OpenAI и LangChain.

## Установка окружения

In [ ]:
import os
from langchain_openai import ChatOpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

from dotenv import load_dotenv
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", base_url=base_url)

# Модель для вычисления семантического сходства
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_similarity(text1, text2):
    """Вычисление семантического сходства двух текстов через косинусное расстояние."""
    embeddings = sentence_model.encode([text1, text2])
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

## Метрики качества промптов

Определим ключевые метрики для оценки эффективности промптов:

In [ ]:
def relevance_score(response, expected_content):
    """Оценка релевантности: семантическое сходство ответа с ожидаемым содержанием."""
    return semantic_similarity(response, expected_content)

def consistency_score(responses):
    """Оценка согласованности: сходство между несколькими ответами на один промпт."""
    if len(responses) < 2:
        return 1.0  # Идеальная согласованность при одном ответе
    similarities = []
    for i in range(len(responses)):
        for j in range(i+1, len(responses)):
            similarities.append(semantic_similarity(responses[i], responses[j]))
    return np.mean(similarities)

def specificity_score(response):
    """Оценка специфичности: доля уникальных слов в ответе."""
    words = response.split()
    unique_words = set(words)
    return len(unique_words) / len(words) if words else 0

## Ручная оценка

Ручная оценка предполагает, что человек анализирует пару «промпт — ответ». Создадим функцию для имитации этого процесса:

In [ ]:
def manual_evaluation(prompt, response, criteria):
    """Имитация ручной оценки пары «промпт — ответ»."""
    print(f"Промпт: {prompt}")
    print(f"Ответ: {response}")
    print("\nКритерии оценки:")
    for criterion in criteria:
        score = float(input(f"Оценка по критерию «{criterion}» (0-10): "))
        print(f"{criterion}: {score}/10")
    print("\nДополнительные комментарии:")
    comments = input("Введите комментарий: ")
    print(f"Комментарий: {comments}")

# Пример использования
prompt = "Объясни простыми словами, что такое дифференцированное обучение."
response = llm.invoke(prompt).content
criteria = ["Ясность", "Точность", "Простота"]
manual_evaluation(prompt, response, criteria)

## Автоматизированная оценка

Реализуем автоматические методы оценки промптов:

In [ ]:
def automated_evaluation(prompt, response, expected_content):
    """Автоматическая оценка пары «промпт — ответ»."""
    relevance = relevance_score(response, expected_content)
    specificity = specificity_score(response)
    
    print(f"Промпт: {prompt}")
    print(f"Ответ: {response}")
    print(f"\nРелевантность: {relevance:.2f}")
    print(f"Специфичность: {specificity:.2f}")
    
    return {"relevance": relevance, "specificity": specificity}

# Пример использования
prompt = "Какие три основных метода оценки знаний учащихся существуют?"
expected_content = "Три основных метода оценки знаний: формативная (текущая), суммативная (итоговая) и диагностическая оценка."
response = llm.invoke(prompt).content
automated_evaluation(prompt, response, expected_content)

## Сравнительный анализ

Сравним эффективность разных промптов для одной и той же задачи:

In [ ]:
def compare_prompts(prompts, expected_content):
    """Сравнение эффективности нескольких промптов для одной задачи."""
    results = []
    for prompt in prompts:
        response = llm.invoke(prompt).content
        evaluation = automated_evaluation(prompt, response, expected_content)
        results.append({"prompt": prompt, **evaluation})
    
    # Сортировка по релевантности
    sorted_results = sorted(results, key=lambda x: x['relevance'], reverse=True)
    
    print("Результаты сравнения промптов:")
    for i, result in enumerate(sorted_results, 1):
        print(f"\n{i}. Промпт: {result['prompt']}")
        print(f"   Релевантность: {result['relevance']:.2f}")
        print(f"   Специфичность: {result['specificity']:.2f}")
    
    return sorted_results

# Пример использования
prompts = [
    "Перечисли методы оценки знаний учащихся.",
    "Какие основные виды оценивания используются в школе?",
    "Объясни разные подходы к оценке успеваемости."
]
expected_content = "Основные методы оценки знаний: формативная (текущая), суммативная (итоговая) и диагностическая оценка."
compare_prompts(prompts, expected_content)

## Комплексная оценка

Создадим функцию, которая объединяет ручную и автоматическую оценку:

In [ ]:
def evaluate_prompt(prompt, expected_content, manual_criteria=["Ясность", "Точность", "Релевантность"]):
    """Комплексная оценка промпта: автоматическая + ручная."""
    response = llm.invoke(prompt).content
    
    print("Автоматическая оценка:")
    auto_results = automated_evaluation(prompt, response, expected_content)
    
    print("\nРучная оценка:")
    manual_evaluation(prompt, response, manual_criteria)
    
    return {"prompt": prompt, "response": response, **auto_results}

# Пример использования
prompt = "Объясни, что такое формативное оценивание и чем оно отличается от суммативного."
expected_content = "Формативное оценивание — это текущая проверка понимания в процессе обучения, направленная на обратную связь и корректировку, в отличие от суммативного оценивания, которое подводит итог за период."
evaluate_prompt(prompt, expected_content)

## Итеративное улучшение через обратную связь

Оценка ответа полезна сама по себе, но настоящая сила раскрывается, когда мы **замыкаем цикл**: передаём результат оценки обратно в модель и просим её улучшить ответ. Это позволяет автоматически повышать качество без ручной переработки промпта.

Цикл работает так:
1. Получаем ответ модели на промпт.
2. Автоматически оцениваем его (релевантность, специфичность).
3. Формируем промпт-обратную связь: исходный ответ + оценки + инструкция «улучши».
4. Получаем улучшенный ответ и оцениваем снова.
5. Сравниваем метрики до и после.

In [ ]:
def iterative_improve(prompt, expected_content, max_iterations=3, target_relevance=0.85):
    """Итеративное улучшение ответа: оценка → обратная связь → повторная генерация."""
    
    response = llm.invoke(prompt).content
    relevance = relevance_score(response, expected_content)
    specificity = specificity_score(response)
    
    print(f"--- Итерация 0 (исходный ответ) ---")
    print(f"Релевантность: {relevance:.2f} | Специфичность: {specificity:.2f}")
    print(f"Ответ: {response[:300]}...")
    print()
    
    history = [{"iteration": 0, "relevance": float(relevance), "specificity": float(specificity)}]
    
    for i in range(1, max_iterations + 1):
        if relevance >= target_relevance:
            print(f"Целевая релевантность ({target_relevance}) достигнута. Остановка.")
            break
        
        # Формируем промпт с обратной связью
        feedback_prompt = f"""Ты — помощник учителя. Тебе был задан вопрос, и ты дал ответ.
Оцени результат и улучши его.

Исходный вопрос: {prompt}

Твой предыдущий ответ:
{response}

Автоматическая оценка предыдущего ответа:
- Релевантность (семантическое сходство с эталоном): {relevance:.2f} из 1.00
- Специфичность (доля уникальных слов): {specificity:.2f} из 1.00

Что нужно улучшить:
- {"Повысь релевантность: ответ должен точнее соответствовать теме вопроса." if relevance < target_relevance else "Релевантность в норме."}
- {"Повысь специфичность: избегай повторов, используй более точные формулировки." if specificity < 0.6 else "Специфичность в норме."}

Дай улучшенный ответ на исходный вопрос:"""
        
        response = llm.invoke(feedback_prompt).content
        relevance = relevance_score(response, expected_content)
        specificity = specificity_score(response)
        
        print(f"--- Итерация {i} ---")
        print(f"Релевантность: {relevance:.2f} | Специфичность: {specificity:.2f}")
        print(f"Ответ: {response[:300]}...")
        print()
        
        history.append({"iteration": i, "relevance": float(relevance), "specificity": float(specificity)})
    
    # Итоговое сравнение
    print("=" * 50)
    print("Сводка по итерациям:")
    for h in history:
        delta_r = h["relevance"] - history[0]["relevance"]
        sign = "+" if delta_r >= 0 else ""
        print(f"  Итерация {h['iteration']}: релевантность={h['relevance']:.2f} ({sign}{delta_r:.2f}), специфичность={h['specificity']:.2f}")
    
    return history

### Пример: улучшение ответа о методах оценки знаний

Зададим вопрос об оценке знаний, запустим цикл улучшения и посмотрим, как меняются метрики.

In [ ]:
# Пример 1: цикл улучшения
prompt = "Какие существуют методы оценки знаний учащихся и в чём их различия?"
expected = (
    "Основные методы оценки знаний: формативная оценка (текущая обратная связь в процессе обучения), "
    "суммативная оценка (итоговая проверка за период) и диагностическая оценка "
    "(выявление пробелов перед началом обучения). Формативная направлена на корректировку процесса, "
    "суммативная — на фиксацию результата, диагностическая — на определение стартового уровня."
)

history = iterative_improve(prompt, expected, max_iterations=3, target_relevance=0.85)

### Пример: улучшение ответа о мотивации учащихся

Попробуем тот же цикл на другом вопросе — о способах повышения мотивации.

In [ ]:
# Пример 2: другой вопрос
prompt_2 = "Как повысить учебную мотивацию школьников?"
expected_2 = (
    "Для повышения мотивации школьников эффективны: связь учебного материала с реальной жизнью, "
    "предоставление выбора в заданиях, своевременная позитивная обратная связь, "
    "использование игровых элементов (геймификация), постановка достижимых целей "
    "и создание безопасной среды, где ошибки воспринимаются как часть обучения."
)

history_2 = iterative_improve(prompt_2, expected_2, max_iterations=3, target_relevance=0.80)